# Clustering

#### Internet Analytics - Lab 3

**Group**: K

**Names**:

- Mathieu Sauser
- Luca Mouchel
- Heikel Jebali
- Jérémy Chaverot

### Exercise 3.12

In [1]:
import json
import matplotlib.pyplot as plt
import numpy as np
import pickle
from sklearn.cluster import KMeans

from bokeh.io import show, output_notebook
from bokeh.plotting import figure, ColumnDataSource
from bokeh.models import HoverTool, SaveTool, ResetTool, PanTool, BoxZoomTool, WheelZoomTool, UndoTool

from itertools import combinations

from operator import itemgetter

%matplotlib inline
plt.style.use("ggplot")

/opt/anaconda3.7/lib/python3.7/site-packages/sklearn/utils/validation.py:37: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  LARGE_SPARSE_SUPPORTED = LooseVersion(scipy_version) >= '0.14.0'


In [2]:
# Retrieve from disk the dictionary created in the Dimension Reduction lab question 3.2
tagNewCoordinates = None
with open("tagNewCoordinates.txt", "rb") as fp:
    tagNewCoordinates = json.load(fp)
    
data = [list(v) for k, v in list(tagNewCoordinates.items())]

# tagNewCoordinates = np.load('./tagNewCoordinates.npy', allow_pickle=True).item()
# data = [(list(values)) for key, values in list(tagNewCoordinates.items())]

In [3]:
def plotData(kMeans, dim1, dim2):
    name = HoverTool(tooltips=[('Name:','@name'),])
    tools = [name, SaveTool(), ResetTool(), PanTool(), BoxZoomTool(), WheelZoomTool(), UndoTool()]
    plot = figure(plot_width=800, plot_height=400, tools=tools, title=f'Principle directions {dim1} and {dim2}.')
    
    clusters = kMeans.cluster_centers_
    colors = ["red", "green", "orange", "yellow", "purple"]
    
    dataSource = ColumnDataSource(
        data = {
            'dim1': [x[dim1] for x in data],
            'dim2': [x[dim2] for x in data],
            'name': list(tagNewCoordinates.keys()),
            'color': [colors[cluster] for cluster in kMeans.labels_]
        }
    )
    plot.circle('dim1','dim2', source=dataSource, size=5, alpha=0.4, color='color')
    
    centersSource = ColumnDataSource(
        data = {
            'dim1': [x[dim1] for x in clusters],
            'dim2': [x[dim2] for x in clusters],
            'name': ['cluster_center' for i in range(5)]
        }
    )
    plot.triangle('dim1', 'dim2', source=centersSource, size=10, alpha=0.6, color='black')

    show(plot, notebook_handle=True)

In [4]:
dims = combinations(range(5), 2)

kMeans = KMeans(n_clusters=5).fit(data)
for dim in dims:
    plot = plotData(kMeans, dim[0], dim[1])
    output_notebook()

Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

It seems that directions 1 and 4 separate the data well

### Exercise 3.13

In [5]:
data = sc.textFile("/ix/ml-20m/movies.txt").map(lambda video: json.loads(video))
# for d in data.take(3):
#    print(d)
    
print('\n---------------------------------------------------\n')

idGenres = data.map(itemgetter('movieId', 'genres'))
# for genre in idGenres.take(3):
#    print(genre)


---------------------------------------------------



In [6]:
with open('./most-rated.pickle', 'rb') as f:
    mostRated = pickle.load(f)
    
# for i in range(3):
#    print(mostRated[i])

In [7]:
mostRatedIds = [movie[0] for movie in mostRated]
moviesInfo = sc.parallelize(mostRated).join(id_genres)
# for i in id_name_genres.take(3):
#    print(i)
    
# print('\n-------------------------\n')
    
temp = moviesInfo.map(lambda info: (info[0], set(info[1][1]))).collect()
moviesToGenres = {item[0]: item[1] for item in temp}
 
# j = 0
# for key, val in moviesToGenres.items():
#    if j < 3:
#        print(key, val)
#    j += 1

NameError: name 'id_genres' is not defined

In [ ]:
def JacDist(A, B):
    numerator = len(moviesToGenres[A].intersection(moviesToGenres[B]))
    denominator = len(moviesToGenres[A].union(moviesToGenres[B]))
    
    return 1 - numerator / denominator

def kMedioids(X, k):
    medioids = np.random.choice(X, k, replace=False)
    nextMedioids = []
    C = [set() for _ in range(k)]
    
    while not np.array_equal(nextMedioids, medioids):
        if len(nextMedioids) > 0:
            medioids = nextMedioids
        
        for x in X:
            distMin = float('inf')
            argMin = None
            
            for i, medioid in enumerate(medioids):
                d = Jac_dist(x, medioid)
                if d < distMin:
                    distMin = d
                    argMin = i
                    
            C[argMin].add(x)

        for cluster in C:
            sumMin = float('inf')
            argMin = None
            
            for x in cluster:
                sumDist = 0
                for y in cluster:
                    sumDist += JacDist(x, y)
                    
                if sumDist < sumMin:
                    sumMin = sumDist
                    argMin = x
                    
            nextMedioids.append(argMin)
            
    return C

In [ ]:
clusters = kMedioids(mostRatedIds, 2)
genresPerVideo1 = [moviesToGenres[movieId] for movieId in clusters[0]]
genresPerVideo2 = [moviesToGenres[movieId] for movieId in clusters[1]]

In [ ]:
def createFreqs(genresPerVideo):
    genreToFreq = {}
    
    for genres in genresPerVideo:
        for genre in genres:
            if genre not in genreToFreq.keys():
                genreToFreq[genre] = 1
            else:
                genreToFreq[genre] += 1
                
    return genreToFreq

In [ ]:
genreToFreq1 = createFreqs(genresPerVideo1)
genreToFreq2 = createFreqs(genresPerVideo2)

In [ ]:
plt.bar(genreToFreq1.keys(), genreToFreq1.values(), color='red')
plt.xticks(rotation='vertical')
plt.title("Frequency of each genre in cluster 1")
plt.show()

In [ ]:
plt.bar(genreToFreq2.keys(), genreToFreq2.values(), color='red')
plt.xticks(rotation='vertical')
plt.title("Frequency of each genre in cluster 2")
plt.show()

The first cluster clearly contains more videos about action, thriller and dram while the second one is more about comedy.